# Building Custom Ansible Modules in Python

Have you ever wanted to type

`ansible -m my-really-cool-custom-module -a 'param1=foo param2=bar'`

... and have it actually work?

Then this section may be for you!

(much of this material is taken from Redhat's [Ansible Module Development Walkthrough][1])

[1]: http://docs.ansible.com/ansible/latest/dev_guide/developing_modules_general.html

*Don't* create a custom module until

- you've looked through all the built-in Ansible modules and can't find something that does what you want
- you've decided that using the `raw` and `script` modules is not powerful enough for what you need
- you know some Python

# Environment setup

- Clone the Ansible repository: `$ git clone https://github.com/ansible/ansible.git`
- Change directory into the repository root dir: `$ cd ansible`
- Create a virtual environment: `$ python3 -m venv venv` (or for Python 2 $ `virtualenv venv`. Note, this requires you to install the virtualenv package: `$ pip install virtualenv`)
- Activate the virtual environment: `$ . venv/bin/activate`
- Install development requirements: `$ pip install -r requirements.txt`
- Run the environment setup script for each new dev shell process: `$ . hacking/env-setup`



In [3]:
cd ~/src/Classes/Ansible

/Users/rick446/src/Classes/Ansible


In [ ]:
%%bash
git clone https://github.com/ansible/ansible.git
cd ansible
python3 -m venv venv
. venv/bin/activate
pip install -r requirements.txt
. hacking/env-setup